In [112]:
import datetime as dt
import numpy as np
import pandas as pd

from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [74]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Create our session (link) from Python to the DB
session = Session(engine)

Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [175]:
Samples = Base.classes.samples
Otu = Base.classes.otu
S_meta = Base.classes.samples_metadata

In [76]:
Base.classes.values()

[sqlalchemy.ext.automap.otu,
 sqlalchemy.ext.automap.samples,
 sqlalchemy.ext.automap.samples_metadata]

In [126]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)


In [127]:
#################################################
# Flask Routes
#################################################

@app.route("/")
def home():
    """Render Home Page."""
    return render_template("index.html")

In [129]:
@app.route('/names')
def names():
    inspector = inspect(engine)
    columns = inspector.get_columns('samples')
        
    names = []

    for column in columns:
        if "BB_" not in column["name"]:
            continue
        else:
            names.append(column["name"])

    return jsonify(names)

In [148]:
otu_columns = inspector.get_columns('otu')

for c in otu_columns:
    print(c['name'], c["type"])

otu_id INTEGER
lowest_taxonomic_unit_found TEXT


In [154]:
results = session.query(Otu.lowest_taxonomic_unit_found).all()
desc = [result[0] for result in results]
print (desc)

['Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus', 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus', 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum', 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter', 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter', 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria', 'Bacteria

In [132]:
columns = inspector.get_columns('samples_metadata')
for c in columns:
    print(c['name'], c["type"])

SAMPLEID INTEGER
EVENT TEXT
ETHNICITY TEXT
GENDER TEXT
AGE INTEGER
WFREQ INTEGER
BBTYPE TEXT
LOCATION TEXT
COUNTRY012 TEXT
ZIP012 INTEGER
COUNTRY1319 TEXT
ZIP1319 INTEGER
DOG TEXT
CAT TEXT
IMPSURFACE013 INTEGER
NPP013 FLOAT
MMAXTEMP013 FLOAT
PFC013 FLOAT
IMPSURFACE1319 INTEGER
NPP1319 FLOAT
MMAXTEMP1319 FLOAT
PFC1319 FLOAT


In [206]:
sample = 'BB_940'

results = session.query(S_meta.AGE, S_meta.BBTYPE, S_meta.ETHNICITY, S_meta.LOCATION, S_meta.SAMPLEID)\
                 .filter(S_meta.SAMPLEID == int(sample[3:])).all()
print (results)

[(24, 'I', 'Caucasian', 'Beaufort/NC', 940)]


In [207]:
wash_number = session.query(S_meta.WFREQ).filter(S_meta.SAMPLEID == int(sample[3:])).all()

print (wash_number)


[(2,)]


In [228]:
sample_columns = inspector.get_columns('samples')

for c in sample_columns:
    print(c['name'], c["type"])

otu_id INTEGER
BB_940 INTEGER
BB_941 INTEGER
BB_943 INTEGER
BB_944 INTEGER
BB_945 INTEGER
BB_946 INTEGER
BB_947 INTEGER
BB_948 INTEGER
BB_949 INTEGER
BB_950 INTEGER
BB_952 INTEGER
BB_953 INTEGER
BB_954 INTEGER
BB_955 INTEGER
BB_956 INTEGER
BB_958 INTEGER
BB_959 INTEGER
BB_960 INTEGER
BB_961 INTEGER
BB_962 INTEGER
BB_963 INTEGER
BB_964 INTEGER
BB_966 INTEGER
BB_967 INTEGER
BB_968 INTEGER
BB_969 INTEGER
BB_970 INTEGER
BB_971 INTEGER
BB_972 INTEGER
BB_973 INTEGER
BB_974 INTEGER
BB_975 INTEGER
BB_978 INTEGER
BB_1233 INTEGER
BB_1234 INTEGER
BB_1235 INTEGER
BB_1236 INTEGER
BB_1237 INTEGER
BB_1238 INTEGER
BB_1242 INTEGER
BB_1243 INTEGER
BB_1246 INTEGER
BB_1253 INTEGER
BB_1254 INTEGER
BB_1258 INTEGER
BB_1259 INTEGER
BB_1260 INTEGER
BB_1264 INTEGER
BB_1265 INTEGER
BB_1273 INTEGER
BB_1275 INTEGER
BB_1276 INTEGER
BB_1277 INTEGER
BB_1278 INTEGER
BB_1279 INTEGER
BB_1280 INTEGER
BB_1281 INTEGER
BB_1282 INTEGER
BB_1283 INTEGER
BB_1284 INTEGER
BB_1285 INTEGER
BB_1286 INTEGER
BB_1287 INTEGER
BB_1288 IN

In [277]:
conn = engine.connect()

df = pd.read_sql('samples', conn)

In [335]:
sample = "BB_940"

sorted_df = df.sort_values(sample, ascending=False)

In [336]:
samples_df = [{"otu_ids": sorted_df["otu_id"].tolist()},
              {"sample_values": sorted_df[sample].tolist()}]


In [337]:
samples_df

[{'otu_ids': [1167,
   2859,
   482,
   2264,
   41,
   1189,
   352,
   189,
   2318,
   1977,
   3450,
   874,
   1959,
   2191,
   1950,
   2077,
   2275,
   944,
   2184,
   2244,
   2024,
   2811,
   2419,
   2782,
   165,
   2011,
   2247,
   2396,
   2964,
   830,
   1795,
   2722,
   2178,
   2908,
   307,
   1193,
   2167,
   1208,
   2039,
   1274,
   2739,
   2737,
   907,
   833,
   2335,
   2936,
   1962,
   2186,
   1314,
   357,
   725,
   2110,
   2571,
   258,
   121,
   1960,
   1169,
   259,
   342,
   1232,
   2188,
   2546,
   340,
   1968,
   170,
   2491,
   1505,
   2291,
   1497,
   412,
   1503,
   2475,
   2235,
   159,
   2065,
   513,
   2342,
   2350,
   2483,
   1498,
   198,
   244,
   1174,
   2949,
   2062,
   3427,
   185,
   2136,
   1179,
   2054,
   1115,
   2633,
   791,
   841,
   3439,
   1155,
   235,
   246,
   2603,
   2917,
   2968,
   3453,
   2915,
   2093,
   226,
   821,
   2114,
   2084,
   2115,
   1159,
   2080,
   794,
   2068,
   20

In [ ]:
otu_ids = session.query(Samples.otu_id).all()
sample_df = [{"otu_ids": []},
             {"sample_values": []}]
sample_df